# Cyclistic Bike-Share Analysis  
## Data Cleaning & Feature Engineering

This notebook focuses on cleaning the raw trip data and creating derived features required for meaningful analysis of rider behaviour.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
DATA_PATH = Path("../data/raw")

files = sorted(DATA_PATH.glob("*.csv"))

df = pd.concat(
    [pd.read_csv(file) for file in files],
    ignore_index=True
)

df.shape

(5552994, 13)

In [3]:
df["started_at"] = pd.to_datetime(df["started_at"], errors="coerce")
df["ended_at"] = pd.to_datetime(df["ended_at"], errors="coerce")

df[["started_at", "ended_at"]].dtypes

started_at    datetime64[ns]
ended_at      datetime64[ns]
dtype: object

In [4]:
df["ride_duration_minutes"] = (
    (df["ended_at"] - df["started_at"])
    .dt.total_seconds()
    / 60
)

df["ride_duration_minutes"].describe()

count    5.552994e+06
mean     1.602772e+01
std      5.511650e+01
min     -5.479480e+01
25%      5.394933e+00
50%      9.426058e+00
75%      1.656325e+01
max      1.574900e+03
Name: ride_duration_minutes, dtype: float64

## Ride Duration Feature

- Ride duration was calculated as the difference between end and start timestamps.
- Duration is expressed in minutes for easier interpretation.
- Negative or zero durations indicate invalid or erroneous records and will be removed.

In [5]:
initial_rows = df.shape[0]

df_clean = df[
    (df["ride_duration_minutes"] > 0) &
    (df["ride_duration_minutes"] <= 1440)
].copy()

final_rows = df_clean.shape[0]

initial_rows, final_rows

(5552994, 5547380)

In [6]:
removed_pct = (initial_rows - final_rows) / initial_rows * 100
round(removed_pct, 2)

0.1

- Approximately 0.1% of total rides were removed during duration-based validation.

## Data Cleaning: Ride Duration Validation

To ensure meaningful analysis, the following records were removed:

- Rides with zero or negative duration
- Rides longer than 24 hours (1,440 minutes)

These records likely represent data errors or operational anomalies rather than typical customer behavior. Removing them improves data quality while preserving the integrity of the analysis.

In [7]:
df_clean["ride_duration_minutes"].describe()

count    5.547380e+06
mean     1.453468e+01
std      2.869838e+01
min      7.666667e-04
25%      5.391483e+00
50%      9.416533e+00
75%      1.652962e+01
max      1.439976e+03
Name: ride_duration_minutes, dtype: float64

In [8]:
df_clean["ride_date"] = df_clean["started_at"].dt.date
df_clean["ride_month"] = df_clean["started_at"].dt.month
df_clean["ride_day_of_week"] = df_clean["started_at"].dt.day_name()
df_clean["ride_hour"] = df_clean["started_at"].dt.hour

In [9]:
df_clean[[
    "started_at",
    "ride_month",
    "ride_day_of_week",
    "ride_hour"
]].head()

,started_at,ride_month,ride_day_of_week,ride_hour
0,2025-01-21 17:23:54.538,1,Tuesday,17
1,2025-01-11 15:44:06.795,1,Saturday,15
2,2025-01-02 15:16:27.730,1,Thursday,15
3,2025-01-23 08:49:05.814,1,Thursday,8
4,2025-01-16 08:38:32.338,1,Thursday,8


In [10]:
df_clean["is_weekend"] = df_clean["ride_day_of_week"].isin(
    ["Saturday", "Sunday"]
)

In [11]:
df_clean["is_weekend"].value_counts(normalize=True).round(3)

is_weekend
False    0.716
True     0.284
Name: proportion, dtype: float64

## Time-based Feature Engineering

The following time-related features were derived from the ride start timestamp:

- Month of ride
- Day of the week
- Hour of the day
- Weekend indicator (Saturday/Sunday)

These features enable analysis of temporal riding patterns and comparison between casual riders and annual members.

In [12]:
df_clean.isnull().sum().sort_values(ascending=False).head(10)

end_station_name         1237830
end_station_id           1237830
start_station_name       1184669
start_station_id         1184669
end_lat                      159
end_lng                      159
ride_id                        0
ride_duration_minutes          0
ride_hour                      0
ride_day_of_week               0
dtype: int64